# Object following without PD controller
## Chasing blue color
### 2019.11.26.Tue

In [1]:
from jetbot import Robot
from jetbot import Camera
from jetbot import bgr8_to_jpeg
import numpy as np
import cv2
import threading
from time import sleep
import queue

In [2]:
def ImgProcessing(camera, imgQueue, redQueue, ROILeftUpX, ROILeftUpY, ROIRightBottomX, ROIRightBottomY):
    while True:
        img = camera.value
        hsvArray = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        cropped = img[ROILeftUpY:ROIRightBottomY, ROILeftUpX:ROIRightBottomX]
        markedImageArray = img
        croppedHSV = hsvArray[ROILeftUpY:ROIRightBottomY, ROILeftUpX:ROIRightBottomX]

        # mark ROI Region Of Interest (in the order of BGR, not RGB)
        borderWIDTH = 1
        # left border
        markedImageArray[ROILeftUpY:ROIRightBottomY, ROILeftUpX:ROILeftUpX+borderWIDTH, 0] = 0
        markedImageArray[ROILeftUpY:ROIRightBottomY, ROILeftUpX:ROILeftUpX+borderWIDTH, 1] = 0
        markedImageArray[ROILeftUpY:ROIRightBottomY, ROILeftUpX:ROILeftUpX+borderWIDTH, 2] = 255
        # top border
        markedImageArray[ROILeftUpY:ROILeftUpY+borderWIDTH, ROILeftUpX:ROIRightBottomX, 0] = 0
        markedImageArray[ROILeftUpY:ROILeftUpY+borderWIDTH, ROILeftUpX:ROIRightBottomX, 1] = 0
        markedImageArray[ROILeftUpY:ROILeftUpY+borderWIDTH, ROILeftUpX:ROIRightBottomX, 2] = 255
        # right border
        markedImageArray[ROILeftUpY:ROIRightBottomY, ROIRightBottomX-borderWIDTH:ROIRightBottomX, 0] = 0
        markedImageArray[ROILeftUpY:ROIRightBottomY, ROIRightBottomX-borderWIDTH:ROIRightBottomX, 1] = 0
        markedImageArray[ROILeftUpY:ROIRightBottomY, ROIRightBottomX-borderWIDTH:ROIRightBottomX, 2] = 255
        # bottom border
        markedImageArray[ROIRightBottomY-borderWIDTH:ROIRightBottomY, ROILeftUpX:ROIRightBottomX, 0] = 0
        markedImageArray[ROIRightBottomY-borderWIDTH:ROIRightBottomY, ROILeftUpX:ROIRightBottomX, 1] = 0
        markedImageArray[ROIRightBottomY-borderWIDTH:ROIRightBottomY, ROILeftUpX:ROIRightBottomX, 2] = 255

        # YELLOW - find column with the most yellow pixels
        # Yellow H value of HSV color space: 60 in range of [0,360], 30 in range of [0,180] in openCV
        # Yellow Range: H:[25,35] S:[100,255] V:[100,255]
        maskYellow = ((20 <= croppedHSV[:,:,0]) & (croppedHSV[:,:,0] <= 40)) \
                        & (croppedHSV[:,:,1] >= 50) & (croppedHSV[:,:,2] >= 50)
        yellows = maskYellow.sum(axis=0)
        laneYellow = yellows.argmax()

        # WHITE - find column with the most white pixels
        # White Range: H:[0,255] S:[0,255] V:[255,255]
        # if the saturation is low and the value is high
        maskWhite = (croppedHSV[:,:,0] <= 0) \
                   & (croppedHSV[:,:,1] <= 0) & (croppedHSV[:,:,2] >= 255)
        whites = maskWhite.sum(axis=0)
        laneWhite = whites.argmax()
#         laneWhite = 0
        # GREEN - find column with the most green pixels
        # Green H value of HSV color space: 120 in range of [0,360], 60 in range of [0,180] in openCV
        # Green Range: H:[55,65] S:[100,255] V:[100,255]
        maskGreen = (40 <= croppedHSV[:,:,0]) & (croppedHSV[:,:,0] <= 80) \
                        & (croppedHSV[:,:,1] >= 100) & (croppedHSV[:,:,2] >= 100)
        greens = maskGreen.sum(axis=0)
        laneGreen = greens.argmax()

        # RED - find column with the most red pixels
        # Red H value of HSV color space: 0~10 and  in range of [0,360], 60 in range of [0,180] in openCV
        # Red Range: H:[55,65] S:[100,255] V:[100,255]
        maskRed = (((0 <= croppedHSV[:,:,0]) & (croppedHSV[:,:,0] <= 20)) | \
                   ((160 <= croppedHSV[:,:,0]) & (croppedHSV[:,:,0] <= 180))) & \
                    (croppedHSV[:,:,1] >= 50) & (croppedHSV[:,:,2] >= 50)
        reds = maskRed.sum(axis=0)
        laneRed = reds.argmax()

        # BLUE
        # Blue Range: H:[110,130] S:[50,255] V:[50,255]
        maskBlue = ((100 <= croppedHSV[:,:,0]) & (croppedHSV[:,:,0] <= 140)) & \
                    (croppedHSV[:,:,1] >= 50) & (croppedHSV[:,:,2] >= 50)
        blues = maskBlue.sum(axis=0)
        laneBlue = blues.argmax()
        
        # mark Yellow
        yellowColumn = ROILeftUpX + laneYellow
        markedImageArray[:,yellowColumn-1:yellowColumn, 0] = 0
        markedImageArray[:,yellowColumn-1:yellowColumn, 1] = 255
        markedImageArray[:,yellowColumn-1:yellowColumn, 2] = 255

        # mark White
        whiteColumn = ROILeftUpX + laneWhite
        markedImageArray[:,whiteColumn-1:whiteColumn, 0] = 255
        markedImageArray[:,whiteColumn-1:whiteColumn, 1] = 255
        markedImageArray[:,whiteColumn-1:whiteColumn, 2] = 255

        # mark Green
        greenColumn = ROILeftUpX + laneGreen
        markedImageArray[:,greenColumn-1:greenColumn, 0] = 0
        markedImageArray[:,greenColumn-1:greenColumn, 1] = 255
        markedImageArray[:,greenColumn-1:greenColumn, 2] = 0

        # mark Red
        redColumn = ROILeftUpX + laneRed
        markedImageArray[:,redColumn-1:redColumn, 0] = 0
        markedImageArray[:,redColumn-1:redColumn, 1] = 0
        markedImageArray[:,redColumn-1:redColumn, 2] = 255

        # mark Blue
        blueColumn = ROILeftUpX + laneBlue
        threshold = 10
        if blues[laneBlue] < threshold:
            laneBlue = 0
            blueColumn = 0
        if blueColumn > 0:
            markedImageArray[:,blueColumn-1:blueColumn, 0] = 255
            markedImageArray[:,blueColumn-1:blueColumn, 1] = 0
            markedImageArray[:,blueColumn-1:blueColumn, 2] = 0

#         print("blue:{}  value:{}".format(blueColumn, blues[laneBlue]))
#         print(blueColumn)
#         print(img[:,blueColumn,0])
#         print(blues)
#         print("shape:{}".format(blues.shape))
        
        if not imgQueue.empty():
#             print("thread imgProcessing(): queue not empty, getting out an item...")
            imgQueue.get()
            redQueue.get()
        if imgQueue.empty():
#             print("900", end=" ")
#             print("thread imgProcessing(): queue is empty, putting an image...")
            imgQueue.put(markedImageArray)
            redQueue.put(blueColumn)
#             sleep(0.1)
#         print("Queue size: {}".format(imgQueue.qsize()))
#         time = 2
#         print("wait for {} seconds...".format(time), end=" ")
#         sleep(time)
#         print("resume...")
#         sleep(1)

In [3]:
def savingImg(imgQueue, redQueue, path):
    count = 0
    while True:
        if not imgQueue.empty():
            count += 1
#             print("thread savingImg(): queue not empty, saving...")
#             print("saved{}".format(count), end=" ")
            cv2.imwrite(path, imgQueue.get())
            redQueue.get()
#         else:
#             print("thread savingImg(): queue is empty... not saving...")
#         sleep(0.1)

In [4]:
def driver(robot, imgQueue, redQueue):
    camW = 640
    camH = 480

#     K = 2.0
#     B = 1.2
    centerRef = camW / 2
    centerAct = 0
    changed = 0
    while True:
        if imgQueue.empty():
            pass
#             print("{}".format(-1), end=" ")
#             print("thread driver(): queue is empty... no direction")
        else:
#             print("thread driver(): queue not empty, getting an image...")
            img = imgQueue.get()
            centerAct = redQueue.get()
#             print("{}".format(centerAct), end=" ")
            error = centerAct - centerRef
            print("error:{}".format(error), end=" ")
#             sleep(0.1)

            th = 100
            forwardspeed = 0.25
            turnspeed = 0.2
            if centerAct == 0:
                print("not detected")
                robot.stop()
                continue
            if -th < error and error < th:
#                 robot.stop()
                print("forward")
                robot.forward(forwardspeed)
            elif error < 0:
                print("left")
                robot.left(turnspeed)
            elif error > 0:
                print("right")
                robot.right(turnspeed)

In [5]:
def createCamera(camW, camH):
    try:
        camera = Camera.instance(width=camW, height=camH)
        print("succeeded to get a camera instance")
    except:
        camera = None
        print('Failed to get a camera instance')
    print("wait for 2 seconds for camera wamr-up", end=" ")
    sleep(2)
    print("resume!")
    return camera

In [6]:
def main():
    camW = 640
    camH = 480
    camera = createCamera(camW, camH)
    
    robot = Robot()
    
    ROIWidth = camW
    ROIHeight = 150
    ROILeftUpX = 0
    ROILeftUpY = 150
    # ROILeftUpX = 150
    # ROILeftUpY = 305
    ROIRightBottomX = ROILeftUpX + ROIWidth
    ROIRightBottomY = ROILeftUpY + ROIHeight
    
    imgQueue = queue.Queue(maxsize=1)
    redQueue = queue.Queue(maxsize=1)
    
    path = "./img/aa.jpg"
    tImg = threading.Thread(target = ImgProcessing, \
                          name = "ImgProcessing", \
                          args=(camera, imgQueue, redQueue, \
                                ROILeftUpX, ROILeftUpY, ROIRightBottomX, ROIRightBottomY))
    tImg.start()

    tsaving = threading.Thread(target = savingImg, \
                         name = "savingImgsavingImg", \
                         args=(imgQueue, redQueue, path))
    tsaving.start()

    tRed = threading.Thread(target = driver, \
                         name = "savingImgsavingImg", \
                         args=(robot, imgQueue, redQueue))
    tRed.start()

    
#     tiktok = 0
#     while tiktok < 50:
#         print("tiktok:{}".format(tiktok))
#         tiktok += 1
#         if not imgQueue.empty():
#             print("saving...")
#             cv2.imwrite("./img/aaa.jpg", imgQueue.get())
#         else:
#             print("empty...")

In [7]:
main()

succeeded to get a camera instance
wait for 2 seconds for camera wamr-up resume!


In [8]:
# print(threading.active_count())
# print(threading.enumerate())

In [9]:
# for i in range(2, 11, 1):
#     speed = i / 10.0
#     r.set_motors(speed, speed)
#     sleep(0.1)
# for i in range(10, -1, -1):
#     speed = i / 10.0
#     r.set_motors(speed, speed)
#     sleep(0.1)
    

In [10]:
# from jetbot import Robot
# r=Robot()
# r.stop()

error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0 not detected
error:-320.0